In [16]:
import time
from simpleai.search import SearchProblem, astar
from simpleai.search.viewers import BaseViewer

# =========================================
# Heuristic h1: Đếm số ô sai vị trí
# =========================================
def h_misplaced(state, goal):
    """
    Hàm heuristic H1:
    Đếm số ô không nằm đúng vị trí so với trạng thái goal.
    (Không tính ô trống = 0).
    """
    count = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0 and val != goal[i][j]:
                count += 1
    return count

# =========================================
# Heuristic h2: Khoảng cách Manhattan
# =========================================
def h_manhattan(state, goal):
    """
    Hàm heuristic H2:
    Tính tổng khoảng cách Manhattan từ vị trí hiện tại của mỗi ô đến vị trí trong trạng thái goal.
    (Không tính ô trống = 0).
    """
    # Lưu lại vị trí đúng của từng giá trị trong goal
    goal_positions = {}
    for i in range(3):
        for j in range(3):
            val = goal[i][j]
            goal_positions[val] = (i, j)

    total = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0:
                gi, gj = goal_positions[val]  # Vị trí đúng trong goal
                total += abs(i - gi) + abs(j - gj)  # Tính khoảng cách Manhattan
    return total

# =========================================
# Lớp định nghĩa bài toán 8 puzzle
# =========================================
class EightPuzzleProblem(SearchProblem):
    """
    Định nghĩa lớp EightPuzzleProblem kế thừa từ SearchProblem.
    Chứa các phương thức: actions, result, is_goal, cost, heuristic
    để mô tả bài toán 8 puzzle cho thuật toán A*.
    """
    def __init__(self, initial, goal, heuristic_func):
        super().__init__(initial_state=initial)
        self.goal = goal
        self.heuristic_func = heuristic_func

    def actions(self, state):
        """
        Trả về các hành động có thể thực hiện từ trạng thái hiện tại:
        Di chuyển ô trống (0) lên, xuống, trái, phải (nếu có thể).
        """
        empty_i, empty_j = [(i, j) for i in range(3) for j in range(3) if state[i][j] == 0][0]
        moves = []
        if empty_i > 0:
            moves.append((empty_i - 1, empty_j))  # Di chuyển ô phía trên xuống
        if empty_i < 2:
            moves.append((empty_i + 1, empty_j))  # Di chuyển ô phía dưới lên
        if empty_j > 0:
            moves.append((empty_i, empty_j - 1))  # Di chuyển ô bên trái sang
        if empty_j < 2:
            moves.append((empty_i, empty_j + 1))  # Di chuyển ô bên phải sang
        return moves

    def result(self, state, action):
        """
        Trả về trạng thái mới sau khi áp dụng action:
        Hoán đổi vị trí ô trống với ô được chọn.
        """
        empty_i, empty_j = [(i, j) for i in range(3) for j in range(3) if state[i][j] == 0][0]
        new_state = [list(row) for row in state]
        tile_i, tile_j = action
        # Hoán đổi ô trống và ô chứa số
        new_state[empty_i][empty_j], new_state[tile_i][tile_j] = new_state[tile_i][tile_j], new_state[empty_i][empty_j]
        return tuple(tuple(row) for row in new_state)

    def is_goal(self, state):
        """Kiểm tra xem state hiện tại có phải goal không."""
        return state == self.goal

    def cost(self, state1, action, state2):
        """Chi phí cho mỗi bước di chuyển luôn = 1."""
        return 1

    def heuristic(self, state):
        """Tính giá trị heuristic dựa trên hàm được truyền vào (h1 hoặc h2)."""
        if self.heuristic_func:
            return self.heuristic_func(state, self.goal)
        return 0

# =========================================
# Hàm in trạng thái và kết quả
# =========================================
def print_state(state):
    """In ra trạng thái bàn cờ, ô trống hiển thị là khoảng trắng."""
    for row in state:
        print(" ".join(str(x) if x != 0 else " " for x in row))
    print()

def print_stats(name, result, time_taken, viewer):
    """
    In ra thống kê kết quả tìm kiếm:
    - Chi phí (g)
    - Độ sâu
    - Số nút mở rộng
    - Số nút sinh ra
    - Thời gian chạy
    Đồng thời in đường đi từ trạng thái đầu → goal.
    """
    print(f"--- Kết quả với {name} ---")
    print(f"Chi phí (g): {result.cost}")
    print(f"Độ sâu: {result.depth}")
    print(f"Số nút mở rộng: {viewer.stats.get('nodes_expanded', 0)}")
    print(f"Số nút sinh ra: {viewer.stats.get('nodes_created', 0)}")
    print(f"Thời gian chạy: {time_taken:.6f} giây")

    print("Đường đi:")
    for node in result.path():
        print_state(node[1])
    print("=" * 30)

# =========================================
# Hàm chạy bài toán với cả 2 heuristic
# =========================================
def run():
    # Trạng thái ban đầu (được chọn sẵn):
    initial_state = ((6, 3, 2),
                     (7, 0, 5),
                     (4, 1, 8))
    # Trạng thái mục tiêu:
    goal_state = ((0, 1, 2),
                  (3, 4, 5),
                  (6, 7, 8))

    print("Trạng thái ban đầu:")
    print_state(initial_state)
    print("Trạng thái mục tiêu:")
    print_state(goal_state)

    # ======= Heuristic H1 ============
    viewer_h1 = BaseViewer()  # Theo dõi quá trình tìm kiếm
    problem_h1 = EightPuzzleProblem(initial_state, goal_state, h_misplaced)
    start_h1 = time.time()
    result_h1 = astar(problem_h1, viewer=viewer_h1, graph_search=False)
    time_h1 = time.time() - start_h1
    print_stats("h1", result_h1, time_h1, viewer_h1)

    # ======= Heuristic H2 ============
    viewer_h2 = BaseViewer()
    problem_h2 = EightPuzzleProblem(initial_state, goal_state, h_manhattan)
    start_h2 = time.time()
    result_h2 = astar(problem_h2, viewer=viewer_h2, graph_search=False)
    time_h2 = time.time() - start_h2
    print_stats("h2", result_h2, time_h2, viewer_h2)

    # ======= So sánh ============
    print("=== SO SÁNH ===")
    print(f"H1 - thời gian: {time_h1:.6f}s | chi phí: {result_h1.cost}")
    print(f"H2 - thời gian: {time_h2:.6f}s | chi phí: {result_h2.cost}")
    print(f"Số nút mở rộng: H1 = {viewer_h1.stats.get('nodes_expanded', 0)}, H2 = {viewer_h2.stats.get('nodes_expanded', 0)}")
    print(f"Số nút sinh ra: H1 = {viewer_h1.stats.get('nodes_created', 0)}, H2 = {viewer_h2.stats.get('nodes_created', 0)}")
    print("H2 thường hiệu quả hơn vì cung cấp thông tin chính xác hơn.")

# =========================================
# Chạy chương trình
# =========================================
if __name__ == "__main__":
    run()


Trạng thái ban đầu:
6 3 2
7   5
4 1 8

Trạng thái mục tiêu:
  1 2
3 4 5
6 7 8

--- Kết quả với h1 ---
Chi phí (g): 10
Độ sâu: 10
Số nút mở rộng: 0
Số nút sinh ra: 0
Thời gian chạy: 0.187136 giây
Đường đi:
6 3 2
7   5
4 1 8

6 3 2
7 1 5
4   8

6 3 2
7 1 5
  4 8

6 3 2
  1 5
7 4 8

  3 2
6 1 5
7 4 8

3   2
6 1 5
7 4 8

3 1 2
6   5
7 4 8

3 1 2
6 4 5
7   8

3 1 2
6 4 5
  7 8

3 1 2
  4 5
6 7 8

  1 2
3 4 5
6 7 8

--- Kết quả với h2 ---
Chi phí (g): 10
Độ sâu: 10
Số nút mở rộng: 0
Số nút sinh ra: 0
Thời gian chạy: 0.002011 giây
Đường đi:
6 3 2
7   5
4 1 8

6 3 2
7 1 5
4   8

6 3 2
7 1 5
  4 8

6 3 2
  1 5
7 4 8

  3 2
6 1 5
7 4 8

3   2
6 1 5
7 4 8

3 1 2
6   5
7 4 8

3 1 2
6 4 5
7   8

3 1 2
6 4 5
  7 8

3 1 2
  4 5
6 7 8

  1 2
3 4 5
6 7 8

=== SO SÁNH ===
H1 - thời gian: 0.187136s | chi phí: 10
H2 - thời gian: 0.002011s | chi phí: 10
Số nút mở rộng: H1 = 0, H2 = 0
Số nút sinh ra: H1 = 0, H2 = 0
H2 thường hiệu quả hơn vì cung cấp thông tin chính xác hơn.


In [36]:
from simpleai.search import SearchProblem
from simpleai.search.local import genetic
import random

# ------------------------------------------------------------
# BÀI TOÁN XẾP HẬU 8×8 DÙNG GIẢI THUẬT DI TRUYỀN (GA)
# ------------------------------------------------------------
# Mục tiêu: Đặt 8 quân hậu lên bàn cờ 8×8 sao cho không có 2 quân nào
# ăn được nhau (ngang, dọc, chéo).
# Biểu diễn: Mỗi state là một tuple gồm N số nguyên.
# - Chỉ số (index) = cột
# - Giá trị = hàng mà quân hậu được đặt.
# Ví dụ: (0, 4, 7, 5, 2, 6, 1, 3) nghĩa là:
#   - Cột 0 có hậu ở hàng 0
#   - Cột 1 có hậu ở hàng 4
#   - ...
# ------------------------------------------------------------

class NQueensProblem(SearchProblem):
    def __init__(self, N=8):
        # Số quân hậu (và cũng là kích thước bàn cờ)
        self.N = N
        # Tổng số cặp hậu tối đa không xung đột = C(N, 2)
        self.max_fitness = self.N * (self.N - 1) // 2
        # Gọi constructor cha với state ban đầu ngẫu nhiên
        super().__init__(initial_state=self.generate_random_state())

    # --------------------------
    # Sinh state ban đầu ngẫu nhiên
    # --------------------------
    def generate_random_state(self):
        # Với mỗi cột, đặt hậu ở một hàng ngẫu nhiên
        return tuple(random.randint(0, self.N - 1) for _ in range(self.N))

    # --------------------------
    # Hàm đánh giá (fitness function)
    # --------------------------
    def value(self, state):
        """
        Tính fitness của state = số cặp hậu không tấn công nhau.
        Fitness càng lớn → nghiệm càng tốt.
        """
        conflicts = 0
        # Duyệt từng cặp hậu (i, j) để kiểm tra xung đột
        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Xung đột nếu: cùng hàng hoặc cùng đường chéo
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        # Số cặp không xung đột = tổng tối đa - số xung đột
        return self.max_fitness - conflicts

    # --------------------------
    # Crossover (lai ghép 2 cá thể)
    # --------------------------
    def crossover(self, state1, state2):
        """
        Chọn một điểm cắt ngẫu nhiên.
        Ghép phần đầu của state1 với phần cuối của state2 để tạo cá thể con.
        """
        cut_point = random.randint(1, self.N - 1)
        return state1[:cut_point] + state2[cut_point:]

    # --------------------------
    # Mutation (đột biến)
    # --------------------------
    def mutate(self, state):
        """
        Chọn một cột ngẫu nhiên rồi gán lại hàng mới cho quân hậu ở cột đó.
        """
        col = random.randint(0, self.N - 1)    # chọn ngẫu nhiên một cột
        new_row = random.randint(0, self.N - 1)  # chọn hàng mới
        state_list = list(state)
        state_list[col] = new_row
        return tuple(state_list)

    # --------------------------
    # Actions và result (không dùng trong GA nhưng bắt buộc phải có)
    # --------------------------
    def actions(self, state):
        return []

    def result(self, state, action):
        return state


# ------------------------------------------------------------
# HÀM CHẠY GIẢI THUẬT DI TRUYỀN
# ------------------------------------------------------------
def run_ga():
    N = 8
    problem = NQueensProblem(N)
    
    # Gọi hàm genetic từ simpleai để giải quyết bài toán
    result = genetic(
        problem,
        population_size=100,   # kích thước quần thể (số cá thể mỗi thế hệ)
        mutation_chance=0.05,  # xác suất đột biến (5%)
        iterations_limit=1000  # giới hạn số vòng lặp
    )

    # In kết quả cuối cùng
    print("=== KẾT QUẢ ===")
    print("Cấu hình lời giải:", result.state)
    print("Fitness:", problem.value(result.state), "/", problem.max_fitness)


# ------------------------------------------------------------
# MAIN
# ------------------------------------------------------------
if __name__ == "__main__":
    run_ga()


=== KẾT QUẢ ===
Cấu hình lời giải: (2, 2, 5, 1, 6, 0, 3, 7)
Fitness: 26 / 28


In [44]:
from simpleai.search import SearchProblem
from simpleai.search.local import simulated_annealing, hill_climbing
import random

N = 8  # số quân hậu trên bàn cờ NxN

# ---------------------------
# Định nghĩa bài toán N-Queens
# ---------------------------
class NQueensProblem(SearchProblem):
    def __init__(self):
        # Mỗi state là 1 list độ dài N
        # state[row] = vị trí cột mà quân hậu được đặt ở hàng "row"
        # Ví dụ: state = [0, 4, 7, 5, 2, 6, 1, 3] nghĩa là:
        # - Hàng 0: quân hậu ở cột 0
        # - Hàng 1: quân hậu ở cột 4
        # ...
        initial_state = [random.randint(0, N - 1) for _ in range(N)]
        super().__init__(initial_state)

    # Trả về các hành động có thể thực hiện từ state hiện tại
    # Hành động = (row, col): di chuyển quân hậu ở hàng "row" sang cột "col"
    def actions(self, state):
        actions = []
        for row in range(N):
            for col in range(N):
                if col != state[row]:  # chỉ xét cột khác vị trí hiện tại
                    actions.append((row, col))
        return actions

    # Kết quả sau khi thực hiện 1 hành động
    def result(self, state, action):
        row, col = action
        new_state = list(state)
        new_state[row] = col
        return new_state

    # Hàm đánh giá (fitness function)
    # Giá trị = số cặp quân hậu KHÔNG tấn công nhau
    # Tổng số cặp có thể = C(N,2) = 28 (với N=8)
    def value(self, state):
        non_attacking = 0
        for i in range(N):
            for j in range(i + 1, N):
                # Hai quân hậu KHÔNG tấn công nhau nếu:
                # - Không cùng cột (state[i] != state[j])
                # - Không cùng đường chéo (|col_i - col_j| != |row_i - row_j|)
                if state[i] != state[j] and abs(state[i] - state[j]) != j - i:
                    non_attacking += 1
        return non_attacking


# ---------------------------
# Hàm in bàn cờ
# ---------------------------
def print_board(state):
    for row in range(N):
        # Nếu cột == state[row] thì có quân hậu (Q), ngược lại là ô trống (.)
        line = ["Q" if state[row] == col else "." for col in range(N)]
        print(" ".join(line))
    print()


# ---------------------------
# Thực thi nhiều lần và chọn kết quả tốt nhất
# ---------------------------
def run_experiments(n_runs=10):
    best_sa = None   # state tốt nhất bằng simulated annealing
    best_hc = None   # state tốt nhất bằng hill climbing
    best_sa_value = -1
    best_hc_value = -1

    for run in range(n_runs):
        problem = NQueensProblem()

        # Chạy Simulated Annealing (giới hạn 1000 bước lặp)
        sa_result = simulated_annealing(problem, iterations_limit=1000)
        sa_val = problem.value(sa_result.state)

        # Chạy Hill Climbing (giới hạn 1000 bước lặp)
        hc_result = hill_climbing(problem, iterations_limit=1000)
        hc_val = problem.value(hc_result.state)

        print(f"Run {run + 1}: SA value = {sa_val}, HC value = {hc_val}")

        # Lưu lại nghiệm tốt nhất tìm được của mỗi thuật toán
        if sa_val > best_sa_value:
            best_sa_value = sa_val
            best_sa = sa_result.state

        if hc_val > best_hc_value:
            best_hc_value = hc_val
            best_hc = hc_result.state

    # ---------------------------
    # In ra kết quả tốt nhất sau n_runs
    # ---------------------------
    print("\n=== KẾT QUẢ TỐT NHẤT ===")

    print("\nSimulated Annealing:")
    print_board(best_sa)
    print("Fitness:", best_sa_value, "/ 28")

    print("\nHill Climbing:")
    print_board(best_hc)
    print("Fitness:", best_hc_value, "/ 28")


# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    run_experiments(10)


Run 1: SA value = 27, HC value = 27
Run 2: SA value = 28, HC value = 27
Run 3: SA value = 27, HC value = 27
Run 4: SA value = 28, HC value = 26
Run 5: SA value = 27, HC value = 28
Run 6: SA value = 27, HC value = 27
Run 7: SA value = 28, HC value = 26
Run 8: SA value = 27, HC value = 27
Run 9: SA value = 28, HC value = 27
Run 10: SA value = 27, HC value = 26

=== KẾT QUẢ TỐT NHẤT ===

Simulated Annealing:
. . . . . Q . .
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
. Q . . . . . .
. . . . . . . Q

Fitness: 28 / 28

Hill Climbing:
. . . Q . . . .
. Q . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
Q . . . . . . .
. . Q . . . . .
. . . . . . Q .

Fitness: 28 / 28


In [43]:
# ===============================
# TIC TAC TOE (XO 3x3) với Minimax + Alpha-Beta
# ===============================

from easyAI import TwoPlayerGame, AI_Player, Negamax
from easyAI.Player import Human_Player

# ANSI màu để in đẹp trong terminal
RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
RESET = '\033[0m'


class TicTacToe(TwoPlayerGame):
    def __init__(self, players):
        # Lưu thông tin người chơi (người + AI)
        self.players = players
        self.current_player = 1  # Player 1 (người) đi trước
        self.board = [0] * 9     # Bảng 3x3: 0 = rỗng, 1 = O, 2 = X

    # ===============================
    # Các hàm chính của game
    # ===============================

    # Trả về danh sách các nước đi hợp lệ (các ô còn trống)
    def possible_moves(self):
        return [str(i+1) for i, v in enumerate(self.board) if v == 0]

    # Thực hiện 1 nước đi (đánh vào ô được chọn)
    def make_move(self, move):
        self.board[int(move)-1] = self.current_player

    # Kiểm tra xem người hiện tại có thua không (nghĩa là đối thủ thắng)
    def loss_condition(self):
        # Các tổ hợp thắng: hàng, cột, đường chéo
        combos = [
            [1, 2, 3], [4, 5, 6], [7, 8, 9],   # hàng ngang
            [1, 4, 7], [2, 5, 8], [3, 6, 9],   # cột dọc
            [1, 5, 9], [3, 5, 7]               # đường chéo
        ]
        # Nếu đối thủ chiếm đủ một tổ hợp thì mình thua
        return any(all(self.board[i-1] == self.opponent_index for i in combo) for combo in combos)

    # Điều kiện kết thúc ván cờ: có người thắng hoặc bàn cờ đầy
    def is_over(self):
        return self.loss_condition() or all(v != 0 for v in self.board)

    # Hàm đánh giá (scoring) cho Minimax:
    # - Nếu mình thua => -100 (xấu cho người hiện tại)
    # - Nếu chưa ai thắng => 0
    def scoring(self):
        return -100 if self.loss_condition() else 0

    # ===============================
    # Hiển thị giao diện
    # ===============================

    # Vẽ bàn cờ ra màn hình
    def show(self):
        print(f"\n{BLUE}=== TIC-TAC-TOE 3x3 ==={RESET}")
        for j in range(3):
            row = []
            for i in range(3):
                val = self.board[3*j+i]
                if val == 0:
                    # Nếu ô trống: hiển thị số để người chơi nhập
                    row.append(str(3*j+i+1))
                elif val == 1:
                    row.append(GREEN + "O" + RESET)  # Người chơi
                else:
                    row.append(RED + "X" + RESET)    # AI
            print(" ".join(row))
        print()

    # Thông báo kết quả sau khi game kết thúc
    def announce_winner(self):
        print(f"\n{YELLOW}=== KẾT QUẢ GAME ==={RESET}")
        if self.loss_condition():
            winner = self.opponent_index
            if winner == 1:
                print(f"{GREEN}🎉 Player 1 (O) THẮNG! 🎉{RESET}")
            else:
                print(f"{RED}🎉 Player 2 (X) THẮNG! 🎉{RESET}")
        else:
            print(f"{YELLOW}🤝 HÒA! 🤝{RESET}")


# ===============================
# Chạy chương trình
# ===============================
if __name__ == "__main__":
    print(f"{BLUE}=== WELCOME TO TIC-TAC-TOE ==={RESET}")
    print(f"{GREEN}Bạn là Player 1 (O){RESET}")
    print(f"{RED}AI là Player 2 (X){RESET}")
    print("Nhập số (1-9) để đánh vào ô tương ứng.")
    print("="*40)

    # Negamax(7): AI sử dụng thuật toán Minimax + Alpha-Beta với độ sâu tìm kiếm = 7
    algorithm = Negamax(7)

    # Người chơi (Human) vs AI
    game = TicTacToe([Human_Player(), AI_Player(algorithm)])
    game.play()
    game.announce_winner()


=== WELCOME TO TIC-TAC-TOE ===
Bạn là Player 1 (O)
AI là Player 2 (X)
Nhập số (1-9) để đánh vào ô tương ứng.

=== TIC-TAC-TOE 3x3 ===
1 2 3
4 5 6
7 8 9


Move #1: player 1 plays 1 :

=== TIC-TAC-TOE 3x3 ===
O 2 3
4 5 6
7 8 9


Move #2: player 2 plays 5 :

=== TIC-TAC-TOE 3x3 ===
O 2 3
4 X 6
7 8 9


Move #3: player 1 plays 2 :

=== TIC-TAC-TOE 3x3 ===
O O 3
4 X 6
7 8 9


Move #4: player 2 plays 3 :

=== TIC-TAC-TOE 3x3 ===
O O X
4 X 6
7 8 9


Move #5: player 1 plays 4 :

=== TIC-TAC-TOE 3x3 ===
O O X
O X 6
7 8 9


Move #6: player 2 plays 7 :

=== TIC-TAC-TOE 3x3 ===
O O X
O X 6
X 8 9


=== KẾT QUẢ GAME ===
🎉 Player 2 (X) THẮNG! 🎉
